In [ ]:
from huggingface_hub import notebook_login
import os
notebook_login()
repo_name='wav2vec2-large-xlsr-sw-ASR'


In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# path of zip file in Google Drive
drive_zip_path = "/content/drive/MyDrive/ASR_wav2vec_project/preprocessing/processed_data/cleaned_sw_audio.zip"

# copy zip file to your Colab working space via rsync
!rsync -avP "$drive_zip_path" /content/

extract_folder = "/content/dataset"

os.makedirs(extract_folder, exist_ok=True)
!unzip -q /content/cleaned_sw_audio.zip -d "$extract_folder"

print(f"Unzip done!: {extract_folder}")

sending incremental file list
cleaned_sw_audio.zip
 14,092,624,648 100%   61.58MB/s    0:03:38 (xfr#1, to-chk=0/1)

sent 14,096,065,334 bytes  received 35 bytes  62,788,709.88 bytes/sec
total size is 14,092,624,648  speedup is 1.00
압축이 풀렸습니다: /content/dataset


In [ ]:
model_path = "facebook/wav2vec2-large-xlsr-53"
BASE = '/content/dataset'
CSV = f'{BASE}/manifest_sw.csv' # You need to put manifest_sw.csv manually
AUDIO_DIR = f'{BASE}/cleaned_sw_audio'
VOCAB = f'{BASE}/vocab.json'

In [ ]:
import pandas as pd
df = pd.read_csv(CSV)
df['wav_path'] = df['wav_path'].apply(lambda p: os.path.join(AUDIO_DIR, p.split('\\')[-1]))
# df["wav_path"] = df["wav_path"].apply(lambda p: os.path.join(AUDIO_DIR, os.path.basename(p)))
print(f"Total samples in dataset: {len(df)}")

Total samples in dataset: 120100


In [ ]:
df['wav_path'][0]

'/content/dataset/cleaned_sw_audio/common_voice_sw_29002392.wav'

In [ ]:
from datasets import Dataset, Audio
dataset = Dataset.from_pandas(df[['wav_path', 'transcript']])

In [ ]:
import re

def remove_special_characters(batch, column_names='transcript'):
  chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\(\)]'
  batch[column_names] = re.sub(chars_to_ignore_regex, '', batch[column_names]).lower() + " "
  return batch

dataset = dataset.map(remove_special_characters)

print(len(dataset['wav_path']))

Map:   0%|          | 0/120100 [00:00<?, ? examples/s]

120100


In [ ]:
def extract_all_chars(batch, column_names='transcript'):
  all_text = " ".join(batch[column_names])
  vocab = list(set(all_text))
  # Return a list of vocabularies and all_text, with length equal to the batch size
  batch_size = len(batch[column_names])
  return {"vocab": [vocab], "all_text": [all_text]}

In [ ]:
vocabs = dataset.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=dataset.column_names)

Map:   0%|          | 0/120100 [00:00<?, ? examples/s]

In [ ]:
vocab_list = list(set(vocabs['vocab'][0]))
vocab_dict ={v: k for k, v in enumerate(vocab_list)}
vocab_dict['|'] = vocab_dict[' ']
del vocab_dict[' ']
vocab_dict['[UNK]'] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
vocab_dict

{'ã': 0,
 'z': 1,
 'a': 2,
 'f': 3,
 'x': 4,
 'p': 5,
 'b': 6,
 'ö': 7,
 'ú': 8,
 'š': 9,
 'ụ': 10,
 '”': 11,
 'r': 12,
 't': 13,
 'n': 14,
 'g': 15,
 'å': 16,
 'ñ': 17,
 'µ': 18,
 'm': 19,
 'u': 20,
 'w': 21,
 'ó': 22,
 '°': 23,
 'ï': 24,
 '`': 25,
 'ū': 26,
 '•': 27,
 'q': 28,
 "'": 29,
 'ø': 30,
 'y': 31,
 'ː': 32,
 'o': 33,
 '’': 34,
 'i': 35,
 'l': 36,
 'd': 37,
 'e': 38,
 's': 39,
 'k': 40,
 'c': 41,
 'ë': 42,
 'j': 43,
 '‘': 44,
 'h': 45,
 'v': 46,
 'â': 47,
 '_': 49,
 '|': 48,
 '[UNK]': 50,
 '[PAD]': 51}

In [ ]:
import json
with open(VOCAB, 'w') as vocab_file:
  json.dump(vocab_dict, vocab_file)

In [ ]:
from transformers import Wav2Vec2CTCTokenizer
tokenizer = Wav2Vec2CTCTokenizer(VOCAB,
                                 unk_token="[UNK]",
                                 pad_token="[PAD]",
                                 word_delimiter_token="|")

In [ ]:
tokenizer.push_to_hub(repo_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

CommitInfo(commit_url='https://huggingface.co/jkhyjkhy/wav2vec2-large-xlsr-sw-ASR/commit/4146ef1a6c4d6404a1e3810c1a507e8ce0257a82', commit_message='Upload tokenizer', commit_description='', oid='4146ef1a6c4d6404a1e3810c1a507e8ce0257a82', pr_url=None, repo_url=RepoUrl('https://huggingface.co/jkhyjkhy/wav2vec2-large-xlsr-sw-ASR', endpoint='https://huggingface.co', repo_type='model', repo_id='jkhyjkhy/wav2vec2-large-xlsr-sw-ASR'), pr_revision=None, pr_num=None)

In [ ]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1,
                                             sampling_rate=16000,
                                             padding_value=0.0,
                                             do_normalize=True,
                                             return_attention_mask=False)

In [ ]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor,
                              tokenizer=tokenizer)

In [ ]:
# Split the dataset first
dataset = dataset.train_test_split(test_size=0.1)
dataset = dataset.cast_column("wav_path", Audio(sampling_rate=16000))

def prepare_dataset(batch):
    audio = batch["wav_path"]

    audio_inputs = processor(audio["array"], sampling_rate=audio["sampling_rate"])

    batch["input_values"] = audio_inputs["input_values"][0]
    batch["input_length"] = len(batch["input_values"])
    label_inputs = processor.tokenizer(batch["transcript"])
    batch["labels"] = label_inputs["input_ids"]

    return batch

In [ ]:
embeded_dataset = dataset.map(prepare_dataset, remove_columns=dataset['train'].column_names)

Map:   0%|          | 0/108090 [00:00<?, ? examples/s]

Map:   0%|          | 0/12010 [00:00<?, ? examples/s]

In [ ]:
embeded_dataset

DatasetDict({
    train: Dataset({
        features: ['input_values', 'input_length', 'labels'],
        num_rows: 108090
    })
    test: Dataset({
        features: ['input_values', 'input_length', 'labels'],
        num_rows: 12010
    })
})

In [ ]:
processor.push_to_hub(repo_name)

CommitInfo(commit_url='https://huggingface.co/jkhyjkhy/wav2vec2-large-xlsr-sw-ASR/commit/3943404a6e22f44efc3c1db3934e8ab29f341215', commit_message='Upload processor', commit_description='', oid='3943404a6e22f44efc3c1db3934e8ab29f341215', pr_url=None, repo_url=RepoUrl('https://huggingface.co/jkhyjkhy/wav2vec2-large-xlsr-sw-ASR', endpoint='https://huggingface.co', repo_type='model', repo_id='jkhyjkhy/wav2vec2-large-xlsr-sw-ASR'), pr_revision=None, pr_num=None)

### Training stage

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
!pip install evaluate jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 88.0 MB/s eta 0:00:00


In [ ]:
import evaluate
wer_metric = evaluate.load('wer')

In [ ]:
import numpy as np
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Config

# Get the vocabulary size from the created vocab_dict
vocab_size = len(vocab_dict)

# Load the pre-trained model configuration
config = Wav2Vec2Config.from_pretrained(model_path)

# Update the config with the correct vocabulary size
config.ctc_loss_reduction = "mean"
config.pad_token_id = processor.tokenizer.pad_token_id
config.vocab_size = vocab_size # Set the vocab_size to the actual size of your vocabulary

# Load the model with the updated configuration
model = Wav2Vec2ForCTC.from_pretrained(
    model_path,
    config=config # Pass the updated config to the model
)

model.freeze_feature_extractor()

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--facebook--wav2vec2-large-xlsr-53/snapshots/c3f9d884181a224a6ac87bf8885c84d1cff3384f/config.json
Model config Wav2Vec2Config {
  "activation_dropout": 0.0,
  "adapter_attn_dim": null,
  "adapter_kernel_size": 3,
  "adapter_stride": 2,
  "add_adapter": false,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForPreTraining"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 768,
  "contrastive_logits_temperature": 0.1,
  "conv_bias": true,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_reduction": "sum",
  "ctc_zero_infinity": false,
  "diversity_loss_weight": 0.1,
  "do_stable_layer_norm": true,
  "eos_token_id": 2,
  "feat_extract_activation

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=False,
  per_device_train_batch_size=8,
  eval_strategy="steps",
  num_train_epochs=1,
  fp16=True,
  gradient_checkpointing=True,
  save_steps=1000,
  eval_steps=1000,
  logging_steps=3000,
  learning_rate=1e-4,
  weight_decay=0.005,
  warmup_steps=1000,
  save_total_limit=2,
  push_to_hub=True,
  report_to=None
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
!wandb disabled

W&B disabled.


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=embeded_dataset["train"],
    eval_dataset=embeded_dataset["test"],
    tokenizer=processor.feature_extractor
)

/tmp/ipython-input-99-3619205471.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Using auto half precision backend


In [ ]:
print(len(trainer.train_dataset))
print(len(trainer.eval_dataset))

108090
12010


In [ ]:
from transformers import logging
logging.enable_progress_bar()
logging.set_verbosity_info()

In [ ]:
trainer.train()

The following columns in the Training set don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 108,090
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 13,512
  Number of trainable parameters = 311,281,844
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
A ConfigError was raised whilst setting the number of model parameters in Weights & Biases config.
/usr/local/lib/python3.11/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:170: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular 

Step,Training Loss,Validation Loss,Wer
1000,No log,0.858771,0.766053
2000,No log,0.313900,0.415924
3000,1.953700,0.251402,0.352360
4000,1.953700,0.221701,0.329236
5000,1.953700,0.198609,0.302932
6000,0.363900,0.184668,0.290329
7000,0.363900,0.175448,0.274714
8000,0.363900,0.165705,0.268878
9000,0.303900,0.158704,0.259835
10000,0.303900,0.156547,0.256000


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:170: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:170: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__

TrainOutput(global_step=13512, training_loss=0.6698858759119498, metrics={'train_runtime': 16256.4736, 'train_samples_per_second': 6.649, 'train_steps_per_second': 0.831, 'total_flos': 1.9304665421375644e+19, 'train_loss': 0.6698858759119498, 'epoch': 1.0})

In [ ]:
trainer.push_to_hub()

Saving model checkpoint to wav2vec2-large-xlsr-sw-ASR
Configuration saved in wav2vec2-large-xlsr-sw-ASR/config.json
Model weights saved in wav2vec2-large-xlsr-sw-ASR/model.safetensors
Feature extractor saved in wav2vec2-large-xlsr-sw-ASR/preprocessor_config.json
Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Automatic Speech Recognition', 'type': 'automatic-speech-recognition'}, 'metrics': [{'name': 'Wer', 'type': 'wer', 'value': 0.2453015676202285}]}


Uploading...:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/jkhyjkhy/wav2vec2-large-xlsr-sw-ASR/commit/5e9bec22d9782713ef035eeceba4e1e035891706', commit_message='End of training', commit_description='', oid='5e9bec22d9782713ef035eeceba4e1e035891706', pr_url=None, repo_url=RepoUrl('https://huggingface.co/jkhyjkhy/wav2vec2-large-xlsr-sw-ASR', endpoint='https://huggingface.co', repo_type='model', repo_id='jkhyjkhy/wav2vec2-large-xlsr-sw-ASR'), pr_revision=None, pr_num=None)

### Evaluate

In [ ]:
# Use the existing tokenizer and feature_extractor to create the processor
processor_eval = Wav2Vec2Processor.from_pretrained(repo_name)
model_eval = Wav2Vec2ForCTC.from_pretrained(repo_name)

/usr/local/lib/python3.11/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:61: FutureWarning: Loading a tokenizer inside Wav2Vec2Processor from a config that does not include a `tokenizer_class` attribute is deprecated and will be removed in v5. Please add `'tokenizer_class': 'Wav2Vec2CTCTokenizer'` attribute to either your `config.json` or `tokenizer_config.json` file to suppress this warning: 
  warnings.warn(


OSError: wav2vec2-large-xlsr-sw-ASR does not appear to have a file named preprocessor_config.json. Checkout 'https://huggingface.co/wav2vec2-large-xlsr-sw-ASR/tree/main' for available files.

In [ ]:
def map_to_result(batch):
  with torch.no_grad():
    input_values = torch.tensor(batch["input_values"]).unsqueeze(0)
    logits = model(input_values).logits

  pred_ids = torch.argmax(logits, dim=-1)
  print(pred_ids)
  batch["pred_str"] = processor_eval.batch_decode(pred_ids)[0]
  batch["text"] = processor_eval.decode(batch["labels"], group_tokens=False)

  return batch

In [ ]:
results = embeded_dataset["test"].map(map_to_result, remove_columns=embeded_dataset["test"].column_names)
print("Test WER: {:.3f}".format(wer_metric.compute(predictions=results["pred_str"], references=results["text"])))

In [ ]:
from IPython.display import display, HTML
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(results)

In [ ]:
tokenizer.decode(embeded_dataset["test"]['labels'][1])

'castro alidai kuwa alilazimishwa kuingia katika mikono ya mungao wa usovieti'

In [ ]:
print("Checking the first few samples of the training dataset:")
for i in range(min(5, len(embeded_dataset["train"]))):
    sample = embeded_dataset["train"][i]
    print(f"\nSample {i}:")
    for key, value in sample.items():
        print(f"  Key: {key}, Type: {type(value)}, Length: {len(value) if hasattr(value, '__len__') else 'N/A'}")
        # Optionally print the first few elements of the value if it's a list or array
        if isinstance(value, list) and len(value) > 0:
            print(f"  First 5 elements: {value[:5]}")
        elif hasattr(value, 'shape') and len(value.shape) > 0:
             print(f"  Shape: {value.shape}")

Checking the first few samples of the training dataset:

Sample 0:
  Key: input_values, Type: <class 'list'>, Length: 82448
  First 5 elements: [0.009276865050196648, 0.004471242893487215, 0.0020684318151324987, -0.0005746603128500283, -0.004178876988589764]
  Key: input_length, Type: <class 'int'>, Length: N/A
  Key: labels, Type: <class 'list'>, Length: 79
  First 5 elements: [39, 45, 20, 15, 45]

Sample 1:
  Key: input_values, Type: <class 'list'>, Length: 67680
  First 5 elements: [-0.003200676990672946, -0.0027381370309740305, -0.0018130569951608777, -0.0013505169190466404, 3.7103076465427876e-05]
  Key: input_length, Type: <class 'int'>, Length: N/A
  Key: labels, Type: <class 'list'>, Length: 51
  First 5 elements: [19, 2, 38, 14, 38]

Sample 2:
  Key: input_values, Type: <class 'list'>, Length: 35776
  First 5 elements: [0.0015478429850190878, 0.0020211106166243553, 0.0020211106166243553, 0.0022577443160116673, 0.002494378248229623]
  Key: input_length, Type: <class 'int'>, Len